In [1]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import keras
import pandas as pd
import numpy as np
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,roc_auc_score,f1_score

Using TensorFlow backend.


In [2]:
train= pd.read_csv('./nlp-getting-started/train.csv')
test=pd.read_csv('./nlp-getting-started/test.csv')
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, train.target.values, 
                                                  stratify=train.target.values, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)
xtest=test.text.values

print(xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape,xtest.shape)

(6851,) (762,) (6851,) (762,) (3263,)


In [9]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid) + list(xtest))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtest_seq = token.texts_to_sequences(xtest)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index


ytrain_enc = keras.utils.to_categorical(ytrain)
yvalid_enc = keras.utils.to_categorical(yvalid)

In [4]:
xtest_pad,word_index

(array([[    0,     0,     0, ...,  1591,   131,    97],
        [    0,     0,     0, ...,   664,  1596,   275],
        [    0,     0,     0, ...,   497,   100,    42],
        ...,
        [    0,     0,     0, ...,     1,     2, 29319],
        [    0,     0,     0, ...,     1,     2,  6123],
        [    0,     0,     0, ...,    75,   259,  3090]]),
 {'t': 1,
  'co': 2,
  'http': 3,
  'the': 4,
  'a': 5,
  'to': 6,
  'in': 7,
  'of': 8,
  'and': 9,
  'i': 10,
  'is': 11,
  'for': 12,
  'on': 13,
  'you': 14,
  'my': 15,
  'it': 16,
  'with': 17,
  'that': 18,
  'by': 19,
  'at': 20,
  'this': 21,
  'https': 22,
  'from': 23,
  'are': 24,
  'be': 25,
  'was': 26,
  'amp': 27,
  'have': 28,
  'like': 29,
  'as': 30,
  'up': 31,
  'just': 32,
  'me': 33,
  'but': 34,
  'not': 35,
  'so': 36,
  'your': 37,
  'out': 38,
  'no': 39,
  'will': 40,
  'after': 41,
  'all': 42,
  'fire': 43,
  'an': 44,
  'we': 45,
  'when': 46,
  'if': 47,
  "i'm": 48,
  'has': 49,
  'get': 50,
  'new': 51

In [5]:
# !pip install gensim

In [6]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [7]:
# dir(model)
model['king'].shape


(300,)

In [8]:
from tqdm import tqdm
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    try:
        embedding_vector = model[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue
    

100%|██████████████████████████████████████████████████████████████████████████| 29319/29319 [00:03<00:00, 7723.75it/s]


In [11]:
# model 1 LSTM
from keras.layers import SpatialDropout1D
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [12]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 6851 samples, validate on 762 samples
Epoch 1/100
6851/6851 [==============================] - ETA: 29s - loss: 0.70 - ETA: 22s - loss: 0.69 - ETA: 19s - loss: 0.69 - ETA: 16s - loss: 0.68 - ETA: 14s - loss: 0.67 - ETA: 12s - loss: 0.67 - ETA: 10s - loss: 0.67 - ETA: 8s - loss: 0.6753 - ETA: 7s - loss: 0.672 - ETA: 5s - loss: 0.669 - ETA: 3s - loss: 0.666 - ETA: 2s - loss: 0.665 - ETA: 0s - loss: 0.664 - 22s 3ms/step - loss: 0.6630 - val_loss: 0.6100
Epoch 2/100
6851/6851 [==============================] - ETA: 18s - loss: 0.65 - ETA: 16s - loss: 0.65 - ETA: 15s - loss: 0.64 - ETA: 13s - loss: 0.63 - ETA: 12s - loss: 0.62 - ETA: 10s - loss: 0.61 - ETA: 9s - loss: 0.6139 - ETA: 7s - loss: 0.609 - ETA: 6s - loss: 0.604 - ETA: 5s - loss: 0.601 - ETA: 3s - loss: 0.597 - ETA: 2s - loss: 0.594 - ETA: 0s - loss: 0.590 - 21s 3ms/step - loss: 0.5894 - val_loss: 0.4622
Epoch 3/100
6851/6851 [==============================] - ETA: 17s - loss: 0.59 - ETA: 16s - loss: 0.56 - ETA: 16s - los

6851/6851 [==============================] - ETA: 17s - loss: 0.43 - ETA: 16s - loss: 0.42 - ETA: 15s - loss: 0.43 - ETA: 14s - loss: 0.43 - ETA: 12s - loss: 0.42 - ETA: 11s - loss: 0.42 - ETA: 9s - loss: 0.4223 - ETA: 8s - loss: 0.425 - ETA: 6s - loss: 0.427 - ETA: 5s - loss: 0.429 - ETA: 3s - loss: 0.427 - ETA: 2s - loss: 0.433 - ETA: 0s - loss: 0.434 - 22s 3ms/step - loss: 0.4347 - val_loss: 0.4172
Epoch 21/100
6851/6851 [==============================] - ETA: 20s - loss: 0.46 - ETA: 18s - loss: 0.44 - ETA: 16s - loss: 0.45 - ETA: 14s - loss: 0.45 - ETA: 13s - loss: 0.44 - ETA: 11s - loss: 0.44 - ETA: 10s - loss: 0.44 - ETA: 8s - loss: 0.4436 - ETA: 6s - loss: 0.439 - ETA: 5s - loss: 0.439 - ETA: 3s - loss: 0.439 - ETA: 2s - loss: 0.439 - ETA: 0s - loss: 0.439 - 22s 3ms/step - loss: 0.4409 - val_loss: 0.4058
Epoch 22/100
6851/6851 [==============================] - ETA: 18s - loss: 0.43 - ETA: 16s - loss: 0.42 - ETA: 14s - loss: 0.43 - ETA: 14s - loss: 0.44 - ETA: 12s - loss: 0.44 -

6851/6851 [==============================] - ETA: 18s - loss: 0.35 - ETA: 17s - loss: 0.37 - ETA: 15s - loss: 0.37 - ETA: 13s - loss: 0.38 - ETA: 11s - loss: 0.38 - ETA: 10s - loss: 0.39 - ETA: 9s - loss: 0.3864 - ETA: 7s - loss: 0.387 - ETA: 6s - loss: 0.391 - ETA: 4s - loss: 0.389 - ETA: 3s - loss: 0.389 - ETA: 1s - loss: 0.388 - ETA: 0s - loss: 0.391 - 20s 3ms/step - loss: 0.3901 - val_loss: 0.4114
Epoch 40/100
6851/6851 [==============================] - ETA: 16s - loss: 0.40 - ETA: 14s - loss: 0.39 - ETA: 13s - loss: 0.39 - ETA: 12s - loss: 0.39 - ETA: 11s - loss: 0.40 - ETA: 10s - loss: 0.40 - ETA: 9s - loss: 0.3987 - ETA: 7s - loss: 0.400 - ETA: 6s - loss: 0.398 - ETA: 4s - loss: 0.396 - ETA: 3s - loss: 0.395 - ETA: 1s - loss: 0.393 - ETA: 0s - loss: 0.391 - 20s 3ms/step - loss: 0.3913 - val_loss: 0.4163
Epoch 41/100
6851/6851 [==============================] - ETA: 18s - loss: 0.37 - ETA: 17s - loss: 0.38 - ETA: 15s - loss: 0.38 - ETA: 13s - loss: 0.39 - ETA: 12s - loss: 0.39 -

6851/6851 [==============================] - ETA: 19s - loss: 0.30 - ETA: 17s - loss: 0.32 - ETA: 16s - loss: 0.34 - ETA: 15s - loss: 0.34 - ETA: 13s - loss: 0.33 - ETA: 11s - loss: 0.33 - ETA: 10s - loss: 0.33 - ETA: 8s - loss: 0.3340 - ETA: 7s - loss: 0.338 - ETA: 5s - loss: 0.337 - ETA: 3s - loss: 0.334 - ETA: 2s - loss: 0.333 - ETA: 0s - loss: 0.335 - 22s 3ms/step - loss: 0.3355 - val_loss: 0.4294
Epoch 59/100
6851/6851 [==============================] - ETA: 20s - loss: 0.34 - ETA: 17s - loss: 0.33 - ETA: 15s - loss: 0.32 - ETA: 14s - loss: 0.32 - ETA: 12s - loss: 0.33 - ETA: 11s - loss: 0.33 - ETA: 9s - loss: 0.3359 - ETA: 8s - loss: 0.341 - ETA: 6s - loss: 0.337 - ETA: 5s - loss: 0.340 - ETA: 3s - loss: 0.339 - ETA: 2s - loss: 0.337 - ETA: 0s - loss: 0.337 - 22s 3ms/step - loss: 0.3361 - val_loss: 0.4422
Epoch 60/100
6851/6851 [==============================] - ETA: 18s - loss: 0.33 - ETA: 16s - loss: 0.34 - ETA: 15s - loss: 0.33 - ETA: 14s - loss: 0.32 - ETA: 12s - loss: 0.32 -

6851/6851 [==============================] - ETA: 17s - loss: 0.31 - ETA: 15s - loss: 0.31 - ETA: 13s - loss: 0.33 - ETA: 12s - loss: 0.30 - ETA: 11s - loss: 0.29 - ETA: 9s - loss: 0.2977 - ETA: 8s - loss: 0.296 - ETA: 7s - loss: 0.298 - ETA: 5s - loss: 0.297 - ETA: 4s - loss: 0.295 - ETA: 3s - loss: 0.295 - ETA: 1s - loss: 0.291 - ETA: 0s - loss: 0.294 - 19s 3ms/step - loss: 0.2961 - val_loss: 0.4610
Epoch 78/100
6851/6851 [==============================] - ETA: 16s - loss: 0.32 - ETA: 14s - loss: 0.31 - ETA: 13s - loss: 0.29 - ETA: 12s - loss: 0.29 - ETA: 11s - loss: 0.29 - ETA: 9s - loss: 0.2910 - ETA: 8s - loss: 0.287 - ETA: 7s - loss: 0.289 - ETA: 5s - loss: 0.286 - ETA: 4s - loss: 0.290 - ETA: 3s - loss: 0.287 - ETA: 1s - loss: 0.287 - ETA: 0s - loss: 0.289 - 19s 3ms/step - loss: 0.2894 - val_loss: 0.4702
Epoch 79/100
6851/6851 [==============================] - ETA: 16s - loss: 0.29 - ETA: 15s - loss: 0.28 - ETA: 14s - loss: 0.29 - ETA: 12s - loss: 0.28 - ETA: 11s - loss: 0.28 -

6851/6851 [==============================] - ETA: 18s - loss: 0.26 - ETA: 17s - loss: 0.27 - ETA: 16s - loss: 0.26 - ETA: 14s - loss: 0.26 - ETA: 13s - loss: 0.26 - ETA: 11s - loss: 0.25 - ETA: 10s - loss: 0.25 - ETA: 8s - loss: 0.2597 - ETA: 6s - loss: 0.255 - ETA: 5s - loss: 0.252 - ETA: 3s - loss: 0.250 - ETA: 2s - loss: 0.247 - ETA: 0s - loss: 0.249 - 22s 3ms/step - loss: 0.2480 - val_loss: 0.5501
Epoch 97/100
6851/6851 [==============================] - ETA: 20s - loss: 0.22 - ETA: 18s - loss: 0.21 - ETA: 16s - loss: 0.22 - ETA: 14s - loss: 0.22 - ETA: 13s - loss: 0.23 - ETA: 11s - loss: 0.23 - ETA: 10s - loss: 0.23 - ETA: 8s - loss: 0.2360 - ETA: 7s - loss: 0.237 - ETA: 5s - loss: 0.240 - ETA: 3s - loss: 0.239 - ETA: 2s - loss: 0.238 - ETA: 0s - loss: 0.239 - 23s 3ms/step - loss: 0.2411 - val_loss: 0.5239
Epoch 98/100
6851/6851 [==============================] - ETA: 19s - loss: 0.22 - ETA: 17s - loss: 0.22 - ETA: 16s - loss: 0.23 - ETA: 15s - loss: 0.23 - ETA: 13s - loss: 0.24 -

In [13]:
from sklearn.metrics import roc_auc_score
predictions = model.predict(xvalid_pad)
# print(classification_report(predictions,yvalid))
score=roc_auc_score(yvalid,predictions[:,1])
print("{} score: {}".format("simpleLSTM",score))

simpleLSTM score: 0.8667158775352385


In [ ]:
from sklearn.metrics import f1_score
predictions=model.predict(xvalid_pad)
thresholds=np.arange(0.1,1,0.05)
for threshold in thresholds:
    temp=predictions[:,1].copy()
    temp[temp>=threshold]=1
    temp[temp<threshold]=0
    score=f1_score(yvalid,temp)
    print("threshold {} score: {}".format(threshold,score))
threshold=0.25
testpred=model.predict(xtest_pad)
testpred=testpred[:,1].copy()
testpred[testpred>=threshold]=1
testpred[testpred<threshold]=0
test["simplenn_0.25"]=testpred
test["simplenn_0.25"]=test["simplenn_0.25"].astype(int)
test[["id","simplenn_0.25"]].rename(columns={"simplenn_0.25":"target"}).to_csv("./submissions/submission_nn_1.csv",index=None)
threshold=0.5
testpred=model.predict(xtest_pad)
testpred=testpred[:,1].copy()
testpred[testpred>=threshold]=1
testpred[testpred<threshold]=0
test["simplenn_0.5"]=testpred
test["simplenn_0.5"]=test["simplenn_0.5"].astype(int)
test[["id","simplenn_0.5"]].rename(columns={"simplenn_0.5":"target"}).to_csv("./submissions/submission_nn_2.csv",index=None)
test.head()

In [23]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
from keras.layers import Bidirectional
model2 = Sequential()
model2.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model2.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.8))

model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.8))

model2.add(Dense(2))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model2.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

ValueError: Input 0 is incompatible with layer bidirectional_1: expected ndim=3, found ndim=2

In [ ]:
predictions = model2.predict(xvalid_pad)
# print(classification_report(predictions,yvalid))
score=roc_auc_score(yvalid,predictions[:,1])
print("{} score: {}".format("BiLSTM",score))

In [ ]:
from keras.layers import GRU
model3 = Sequential()
model3.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model3.add(SpatialDropout1D(0.3))
model3.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model3.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model3.add(Dense(1024, activation='relu'))
model3.add(Dropout(0.8))

model3.add(Dense(1024, activation='relu'))
model3.add(Dropout(0.8))

model3.add(Dense(2))
model3.add(Activation('softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model3.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 6851 samples, validate on 762 samples
Epoch 1/100
6851/6851 [==============================] - ETA: 1:54 - loss: 0.699 - ETA: 1:32 - loss: 0.693 - ETA: 1:19 - loss: 0.689 - ETA: 1:10 - loss: 0.684 - ETA: 1:02 - loss: 0.680 - ETA: 55s - loss: 0.679 - ETA: 48s - loss: 0.67 - ETA: 40s - loss: 0.67 - ETA: 32s - loss: 0.66 - ETA: 25s - loss: 0.66 - ETA: 17s - loss: 0.66 - ETA: 10s - loss: 0.66 - ETA: 2s - loss: 0.6608 - 102s 15ms/step - loss: 0.6622 - val_loss: 0.6129
Epoch 2/100
6851/6851 [==============================] - ETA: 1:27 - loss: 0.638 - ETA: 1:23 - loss: 0.630 - ETA: 1:17 - loss: 0.629 - ETA: 1:09 - loss: 0.626 - ETA: 1:02 - loss: 0.619 - ETA: 55s - loss: 0.617 - ETA: 48s - loss: 0.60 - ETA: 40s - loss: 0.61 - ETA: 33s - loss: 0.60 - ETA: 25s - loss: 0.61 - ETA: 18s - loss: 0.60 - ETA: 10s - loss: 0.59 - ETA: 2s - loss: 0.5935 - 107s 16ms/step - loss: 0.5929 - val_loss: 0.4844
Epoch 3/100
6851/6851 [==============================] - ETA: 1:40 - loss: 0.521 - ETA: 1:30 

In [ ]:
from sklearn.metrics import roc_auc_score
predictions = model3.predict(xvalid_pad)
# print(classification_report(predictions,yvalid))
score=roc_auc_score(yvalid,predictions[:,1])
print("{} score: {}".format("GRU",score))

In [ ]:
# import pickle
# pickle.dump(model3,open("./models/model3.p","wb"))